In [1]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr

from sklearn.linear_model import LinearRegression, LogisticRegression
from tqdm import tqdm

## Get Stock List

In [2]:
path = '../data'
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


## Get Data & Modeling

In [3]:
# start_date = '20210104'
start_date = '20200106'
end_date = '20211105'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 44
HOW MANY "Business_days" : (480, 1)


,Date
0,2020-01-06
1,2020-01-07
2,2020-01-08
3,2020-01-09
4,2020-01-10


## Baseline 모델의 구성 소개 ( Sample )

- X : (월 ~ 금) * 43주간
- y : (다음주 월 ~ 금) * 43주간
    - y_0 : 다음주 월요일
    - y_1 : 다음주 화요일
    - y_2 : 다음주 수요일
    - y_3 : 다음주 목요일
    - y_4 : 다음주 금요일


- 이번주 월~금요일의 패턴을 학습해 다음주 월요일 ~ 금요일을 각각 예측하는 모델을 생성
    
- 이 과정을 모든 종목(370개)에 적용

In [4]:
'''
지금 오늘것을 입력해서 내일 종가를 맞추는 모형으로 만듦

- 문제는 지난 1주일(5일) 데이터로 다음 1주일(5일간) 종가를 맞추기
- 현재 하루 짜리 입력을 쓰는 것에서 5일짜리 입력을 쓰도록 수정필요

'''

sample_code = stock_list.loc[0,'종목코드']

sample = fdr.DataReader(sample_code, start = start_date, end = end_date).reset_index()
sample = pd.merge(Business_days, sample, how = 'outer')
sample['weekday'] = sample.Date.apply(lambda x : x.weekday())
sample['weeknum'] = sample.Date.apply(lambda x : x.strftime('%V'))

feature_columns = 'Open	High	Low	Close	Volume	Change	weekday'.split()
for c in feature_columns:
    sample[c] = sample[c].ffill()
    sample[c] = sample[c].bfill()

sample.head()

,Date,Open,High,Low,Close,Volume,Change,weekday,weeknum
0,2020-01-06,54900.0,55600.0,54600.0,55500.0,10278951.0,0.000000,0,02
1,2020-01-07,55700.0,56400.0,55600.0,55800.0,10009778.0,0.005405,1,02
2,2020-01-08,56200.0,57400.0,55900.0,56800.0,23501171.0,0.017921,2,02
3,2020-01-09,58400.0,58600.0,57400.0,58600.0,24102579.0,0.031690,3,02
4,2020-01-10,58800.0,59700.0,58300.0,59500.0,16000170.0,0.015358,4,02


In [5]:
# 모형 1: 5일치 데이터(Open	High	Low	Close	Volume	Change	weekday)로 다음날 값을 예측
# 모형 2: 5일치 데이터(Open	High	Low	Close	Volume	Change	weekday)로 2일 뒤 값을 예측
# 모형 3: 5일치 데이터(Open	High	Low	Close	Volume	Change	weekday)로 3일 뒤 값을 예측
# 모형 4: 5일치 데이터(Open	High	Low	Close	Volume	Change	weekday)로 4일 뒤 값을 예측
# 모형 5: 5일치 데이터(Open	High	Low	Close	Volume	Change	weekday)로 5일 뒤 값을 예측

window_size = 5
X = np.array(
    [sample[feature_columns].iloc[i:i+window_size].to_numpy().reshape(-1) for i in range(len(sample) - 14)]
)
print(X.shape)
pd.DataFrame(X[-1].reshape(5, 7))

(466, 35)


,0,1,2,3,4,5,6
0,70200.0,70300.0,69200.0,70200.0,13486391.0,0.001427,0.0
1,70800.0,71000.0,70400.0,70600.0,9507991.0,0.005698,1.0
2,70700.0,71000.0,70200.0,70300.0,10151638.0,-0.004249,2.0
3,70300.0,70600.0,70000.0,70200.0,10891094.0,-0.001422,3.0
4,70000.0,70700.0,70000.0,70400.0,8395448.0,0.002849,4.0


In [6]:
y_values = [sample['Close'].iloc[5+i:-9+i].to_numpy() for i in range(5)]
print([y.shape for y in y_values])
[y[:5] for y in y_values], [y[-5:] for y in y_values]

[(466,), (466,), (466,), (466,), (466,)]


([array([60000., 60000., 59000., 60700., 61300.]),
  array([60000., 59000., 60700., 61300., 62400.]),
  array([59000., 60700., 61300., 62400., 61400.]),
  array([60700., 61300., 62400., 61400., 62300.]),
  array([61300., 62400., 61400., 62300., 60800.])],
 [array([70600., 70300., 70200., 70400., 70200.]),
  array([70300., 70200., 70400., 70200., 71100.]),
  array([70200., 70400., 70200., 71100., 70100.]),
  array([70400., 70200., 71100., 70100., 70700.]),
  array([70200., 71100., 70100., 70700., 69800.])])

In [7]:
x_public = sample[feature_columns].iloc[-10:-10+window_size].to_numpy().reshape(-1)
print(x_public.shape)
pd.DataFrame(x_public.reshape(5, 7))

(35,)


,0,1,2,3,4,5,6
0,69900.0,70600.0,69500.0,70200.0,10029621.0,-0.002841,0.0
1,70600.0,71500.0,70400.0,71100.0,10528252.0,0.012821,1.0
2,71000.0,71000.0,70000.0,70100.0,10295316.0,-0.014065,2.0
3,69500.0,72200.0,69500.0,70700.0,20644902.0,0.008559,3.0
4,71200.0,71600.0,69700.0,69800.0,17016151.0,-0.012730,4.0


- 예측

In [8]:
model = LinearRegression()

predictions = []
for y_value in y_values :
    model.fit(X,y_value)
    prediction = model.predict(np.expand_dims(x_public,0))
    predictions.append(prediction[0])
predictions

[69810.42851445179,
 69966.17403496984,
 70012.37650528594,
 69876.18596324943,
 69609.71201714051]

- 실제 Public 값

In [9]:
sample['Close'].iloc[-5:].to_numpy()

array([69900., 71500., 70400., 70600., 70200.])

# 전체 모델링

In [10]:
sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path,sample_name))
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-11-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-11-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-11-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-11-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2021-11-29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2021-11-30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2021-12-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2021-12-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2021-12-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
model = LinearRegression()
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date).reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))

    feature_columns = 'Open	High	Low	Close	Volume	Change	weekday'.split()
    for c in feature_columns:
        data[c] = data[c].ffill()
        data[c] = data[c].bfill()


    window_size = 5
    X = np.array(
        [data[feature_columns].iloc[i:i+window_size].to_numpy().reshape(-1) for i in range(len(data) - 14)]
    )
    y_values = [data['Close'].iloc[5+i:-9+i].to_numpy() for i in range(5)]
    x_public = data[feature_columns].iloc[-10:-10+window_size].to_numpy().reshape(-1)


    predictions = []
    for y_value in y_values :
        model.fit(X,y_value)
        prediction = model.predict(np.expand_dims(x_public,0))
        predictions.append(prediction[0])
    sample_submission.loc[:,code] = predictions * 2
sample_submission.isna().sum().sum()

100%|██████████| 370/370 [02:09<00:00,  2.86it/s]


0

In [12]:
sample_submission.columns

Index(['Day', '000060', '000080', '000100', '000120', '000150', '000240',
       '000250', '000270', '000660',
       ...
       '330860', '336260', '336370', '347860', '348150', '348210', '352820',
       '357780', '363280', '950130'],
      dtype='object', length=371)

In [13]:
columns = list(sample_submission.columns[1:])

columns = ['Day'] + [str(x).zfill(6) for x in columns]

sample_submission.columns = columns

In [14]:
sample_submission.to_csv('BASELINE_Linear.csv',index=False)

In [15]:
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,28027.132050,34977.722639,60315.743432,142870.716107,101404.515734,16657.893495,46798.971680,85789.284288,104880.102337,...,49079.579800,52025.375876,82840.981146,37730.735333,25388.743618,51781.422763,332084.818935,265353.824200,27073.073330,17174.583999
1,2021-11-02,28110.237259,35235.103158,60082.666571,143485.402680,102090.292128,16822.463526,46524.364639,85283.869855,105214.121889,...,47886.016247,51259.202516,83201.793893,36784.323358,25265.333125,51386.707927,330853.577491,265451.883247,27327.757695,16850.479549
2,2021-11-03,28208.228363,35083.276875,60140.679364,144817.188163,103599.602867,16736.153640,46198.665146,86646.547181,104307.793338,...,47550.174128,50941.040203,84224.580227,36549.210023,25697.743778,50915.757322,331082.749812,266690.583775,27375.584643,16861.828838
3,2021-11-04,28061.435455,34917.034643,59895.392775,145342.112565,102880.118485,16775.355617,47022.855194,87557.851018,105100.544130,...,47859.466986,51305.994804,83395.174731,37240.334929,25219.698578,50974.687262,333046.211305,268914.176335,27367.878909,17163.340685
4,2021-11-05,28186.762360,34453.511856,59837.716070,145042.047171,102741.829701,16619.099722,47502.122833,87089.567616,105043.581280,...,47978.026600,51260.946287,84100.214711,37221.636134,24958.315031,50858.762415,331131.647119,272054.360361,27256.469600,17013.186482
5,2021-11-29,28027.132050,34977.722639,60315.743432,142870.716107,101404.515734,16657.893495,46798.971680,85789.284288,104880.102337,...,49079.579800,52025.375876,82840.981146,37730.735333,25388.743618,51781.422763,332084.818935,265353.824200,27073.073330,17174.583999
6,2021-11-30,28110.237259,35235.103158,60082.666571,143485.402680,102090.292128,16822.463526,46524.364639,85283.869855,105214.121889,...,47886.016247,51259.202516,83201.793893,36784.323358,25265.333125,51386.707927,330853.577491,265451.883247,27327.757695,16850.479549
7,2021-12-01,28208.228363,35083.276875,60140.679364,144817.188163,103599.602867,16736.153640,46198.665146,86646.547181,104307.793338,...,47550.174128,50941.040203,84224.580227,36549.210023,25697.743778,50915.757322,331082.749812,266690.583775,27375.584643,16861.828838
8,2021-12-02,28061.435455,34917.034643,59895.392775,145342.112565,102880.118485,16775.355617,47022.855194,87557.851018,105100.544130,...,47859.466986,51305.994804,83395.174731,37240.334929,25219.698578,50974.687262,333046.211305,268914.176335,27367.878909,17163.340685
9,2021-12-03,28186.762360,34453.511856,59837.716070,145042.047171,102741.829701,16619.099722,47502.122833,87089.567616,105043.581280,...,47978.026600,51260.946287,84100.214711,37221.636134,24958.315031,50858.762415,331131.647119,272054.360361,27256.469600,17013.186482


In [38]:
public_answer = pd.concat([fdr.DataReader(c, start = start_date, end = end_date)[['Close']].rename(columns={'Close': c}) for c in sample_submission.columns if c != 'Day'], axis=1)
public_answer = public_answer.reset_index().rename(columns={'Date': 'Day'})
public_answer


,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,27850,35200,60000,146000,103000,16600,49750,84300,106500,...,48300,51700,80500,36750,25700,52000,348500,261600,26600,17600
1,2021-11-02,29250,35050,61700,148500,107000,17350,48950,86000,107500,...,50800,54100,81600,35850,25950,52900,348000,258600,27100,18100
2,2021-11-03,30250,34050,61500,145500,110000,17100,50300,85000,105500,...,49450,54600,82700,34700,25450,51200,346500,253700,26750,18300
3,2021-11-04,29450,33800,61100,145000,125500,17750,50600,87000,106000,...,48700,53800,81500,35500,25500,50600,356500,249600,26350,17550
4,2021-11-05,29550,33450,60600,144500,133500,17300,50400,88000,107000,...,49800,53900,85100,35500,25100,50200,383500,247800,26050,17550


In [39]:
public_submission = sample_submission.head(5)
public_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,28027.132050,34977.722639,60315.743432,142870.716107,101404.515734,16657.893495,46798.971680,85789.284288,104880.102337,...,49079.579800,52025.375876,82840.981146,37730.735333,25388.743618,51781.422763,332084.818935,265353.824200,27073.073330,17174.583999
1,2021-11-02,28110.237259,35235.103158,60082.666571,143485.402680,102090.292128,16822.463526,46524.364639,85283.869855,105214.121889,...,47886.016247,51259.202516,83201.793893,36784.323358,25265.333125,51386.707927,330853.577491,265451.883247,27327.757695,16850.479549
2,2021-11-03,28208.228363,35083.276875,60140.679364,144817.188163,103599.602867,16736.153640,46198.665146,86646.547181,104307.793338,...,47550.174128,50941.040203,84224.580227,36549.210023,25697.743778,50915.757322,331082.749812,266690.583775,27375.584643,16861.828838
3,2021-11-04,28061.435455,34917.034643,59895.392775,145342.112565,102880.118485,16775.355617,47022.855194,87557.851018,105100.544130,...,47859.466986,51305.994804,83395.174731,37240.334929,25219.698578,50974.687262,333046.211305,268914.176335,27367.878909,17163.340685
4,2021-11-05,28186.762360,34453.511856,59837.716070,145042.047171,102741.829701,16619.099722,47502.122833,87089.567616,105043.581280,...,47978.026600,51260.946287,84100.214711,37221.636134,24958.315031,50858.762415,331131.647119,272054.360361,27256.469600,17013.186482


In [69]:
y_pred_div_y = (public_submission.drop('Day', axis=1).values - public_answer.drop('Day', axis=1).values) / public_answer.drop('Day', axis=1).values
np.sqrt((y_pred_div_y*y_pred_div_y).mean())

0.11826682257100153